## Use catalog

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment or catalog in lower case')
env = dbutils.widgets.get("env")
print(env)

In [0]:

bronze_path = spark.sql("""describe external location bronze_db;""").select('url').collect()[0][0]
silver_path = spark.sql("""describe external location silver_db;""").select('url').collect()[0][0]
gold_path = spark.sql("""describe external location gold_db;""").select('url').collect()[0][0]

In [0]:
def create_Bronze_schema(environment,path):
    print(f"Using catalog {environment}")
    spark.sql(f"""use catalog {environment}""")
    print(f"creating schema for bronze")
    spark.sql(f"""CREATE SCHEMA IF NOT EXISTS BRONZE MANAGED LOCATION '{path}/bronze';""")

def create_Silver_schema(environment,path):
    print(f"Using catalog {environment}")
    spark.sql(f"""use catalog {environment}""")
    print(f"creating schema for silver")
    spark.sql(f"""CREATE SCHEMA IF NOT EXISTS SILVER MANAGED LOCATION '{path}/silver';""")

def create_Gold_schema(environment,path):
    print(f"Using catalog {environment}")
    spark.sql(f"""use catalog {environment}""")
    print(f"creating schema for gold")
    spark.sql(f"""CREATE SCHEMA IF NOT EXISTS GOLD MANAGED LOCATION '{path}/gold';""")

In [0]:
create_Bronze_schema(env,path=bronze_path)
create_Silver_schema(env,path=silver_path)
create_Gold_schema(env,path=gold_path)

## Creating Bronze Layer Tables

In [0]:
def createTable_rawTraffic(environment):
    print(f'Creating raw_Traffic table in {environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}`.`bronze`.`raw_traffic`
                        (
                            Record_ID INT,
                            Count_point_id INT,
                            Direction_of_travel VARCHAR(255),
                            Year INT,
                            Count_date VARCHAR(255),
                            hour INT,
                            Region_id INT,
                            Region_name VARCHAR(255),
                            Local_authority_name VARCHAR(255),
                            Road_name VARCHAR(255),
                            Road_Category_ID INT,
                            Start_junction_road_name VARCHAR(255),
                            End_junction_road_name VARCHAR(255),
                            Latitude DOUBLE,
                            Longitude DOUBLE,
                            Link_length_km DOUBLE,
                            Pedal_cycles INT,
                            Two_wheeled_motor_vehicles INT,
                            Cars_and_taxis INT,
                            Buses_and_coaches INT,
                            LGV_Type INT,
                            HGV_Type INT,
                            EV_Car INT,
                            EV_Bike INT,
                            Extract_Time TIMESTAMP
                    );""")
    
    print("************************************")

In [0]:
def createTable_rawRoad(environment):
    print(f'Creating raw_roads table in {environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}`.`bronze`.`raw_roads`
                        (
                            Road_ID INT,
                            Road_Category_Id INT,
                            Road_Category VARCHAR(255),
                            Region_ID INT,
                            Region_Name VARCHAR(255),
                            Total_Link_Length_Km DOUBLE,
                            Total_Link_Length_Miles DOUBLE,
                            All_Motor_Vehicles DOUBLE
                    );""")
    
    print("************************************")

In [0]:
createTable_rawRoad(env)
createTable_rawTraffic(env)

In [0]:
def reset_catalog_schemas():

    spark.sql("""drop schema `bronze`;""")
    spark.sql("""drop schema `silver`;""")
    spark.sql("""drop schema `gold`;""")
    return "Dropped all schemas"